# Model Building

In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from typing import List

Set a predetermined seed so all our results can be replicated

In [ ]:
RANDOM_SEED = 1337

# Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
full_data = pd.read_csv('Dataset/clean_data.csv', index_col='Customer_ID')

In [ ]:
#Separate target column with rest of the data
churn_col = full_data['churn'].copy()
full_data = full_data.drop('churn',axis=1)

Since we're finished with EDA in the other notebook, we can start splitting the data into training and testing sets. For models that need validation, we will utilize k-fold CV later on. 

In [8]:
#Separate data for training and testing with 80% for training and 20% testing
#Uses our preselected random seed to results are reproducible 
raw_x_train, raw_x_test, y_train, y_test = train_test_split(
    full_data,
    churn_col,
    test_size=0.2,
    random_state=RANDOM_SEED
)

In [ ]:
with open('columnDescriptions.json','r') as f:
    col_desc = json.load(f)
    
#Shortened descriptions with elipses for plot titles
#Only retains first 20 characters of description then appends with elipses
short_col_desc = dict(zip(
    col_desc.keys(),
    map(lambda desc: 
        desc if len(desc)<20 else f'{desc[:20]}...', col_desc.values()
    )
))

We will use several different algorithms then compare their performance afterwards to determine which is the best to use. The algorithm we will use are: 
- Logistic Regression
- K Nearest Neighbor Classifier
- Random Forest
- XGBoost
- LightGBM

Since the implementations selected for the above algorithms have differing aptitudes for missing values and normalization, we will need different preprocessing pipelines of the data. For example, XGBoost and LightGBM can handle nan values (XGBoost learns whether to split nan values during training, while LightGBM allocates nan values to reduce loss afterwards) while the sklearn implementations of Logisitic regression, KNN classifier, and random forest cannot. 

In [1]:
from model_utils import PipelineFactory

In [71]:
pf = PipelineFactory(full_data)
pca_pipe = pf.create_pipe(pca=True,impute=True,normalize=True)
impute_normalize_pipe = pf.create_pipe(impute=True,normalize=True)
impute_pipe = pf.create_pipe(impute=True,normalize=False)
ohe_pipe = pf.create_pipe(impute=False,normalize=False)

# Model building

In [12]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgbm

## Logistic Regression

In [73]:
pca_pipe

Pipeline(steps=[('impute', SimpleImputer(strategy='most_frequent')),
                ('ScaleNumerical',
                 ColumnTransformer(transformers=[('categorical_vars',
                                                  'passthrough',
                                                  array([ 5, 28, 36, 37, 42, 44, 53, 66, 72, 79], dtype=int64)),
                                                 ('numeric_vars',
                                                  StandardScaler(),
                                                  array([ 0,  1,  2,  3,  4,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 35, 38, 41,
       43, 45, 46, 52, 54...
                                                 ('numeric_vars',
                                                  PCA(random_state=1337),
                                                  array([ 0,  1,  2,  3,  4,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 35, 38, 41,
       43, 45, 46, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67,
       68, 69, 70, 71, 73, 74, 75, 76, 77, 78, 80, 81, 83, 84, 86, 87, 88,
       89, 90, 92, 93, 94, 95], dtype=int64)),
                                                 ('binary_vars', 'passthrough',
                                                  array([ 6, 29, 39, 40, 47, 48, 49, 50, 51, 82, 85, 91], dtype=int64))]))])

In [74]:
# pca_pipe.steps[2][1].fit_transform(
pca_pipe.steps[1][1].fit_transform(
    pca_pipe.steps[0][1].fit_transform(raw_x_train)
)
# )

array([['GREAT LAKES AREA', 'AA', 'Y', ..., True, False, False],
       ['SOUTHWEST AREA', 'AA', 'Y', ..., True, True, True],
       ['CALIFORNIA NORTH AREA', 'BA', 'Y', ..., False, False, False],
       ...,
       ['LOS ANGELES AREA', 'CY', 'Y', ..., False, False, False],
       ['CHICAGO AREA', 'BA', 'Y', ..., False, False, False],
       ['CALIFORNIA NORTH AREA', 'B', 'Y', ..., False, False, False]],
      dtype=object)

In [75]:
x_train = pca_pipe.fit_transform(raw_x_train)
x_test = impute_normalize_pipe.transform(raw_x_test)

In [18]:
# L2 regularized logistic regression
# 5 default chosen regularization strength
# 5 fold CV (80% training 20% validation)
log_reg = LogisticRegressionCV(
    Cs=
    random_state=RANDOM_SEED,
    class_weight='balanced'
)